## Log
With this, can skip 1.Concate fund & 2.Concate Monthly Return
This notebook is created after the presentation, mainly for web scraping, use "Invesco" instead of "Investco"
Also, the old version generates 500 random portfolios, 
    the current version test [0 0.1 0.2 0.3 0.4 0.5 0.6 0.65], to limit the range first

## Install Selenium

In [1]:
!pip install selenium

## Download Chromedriver
https://selenium-python.readthedocs.io/installation.html#drivers

## Folder location 
Where you save the csv file after web scraping

In [2]:
path_dir: str = r"C:\Users\thy10\Desktop\AREIX\MPF Data\Invesco try"
print(path_dir)

C:\Users\thy10\Desktop\AREIX\MPF Data\Invesco try


## Web Scraping

In [3]:
def web_scrap():
    import os 
    from time import sleep
    from selenium import webdriver

    try:
        #delete file in the path 
        for count, filename in enumerate(os.listdir(path_dir)):  
            os.remove(path_dir+"/"+filename)
        
        options = webdriver.ChromeOptions()

        prefs = {"download.default_directory":path_dir}

        options.add_experimental_option('prefs', prefs)
        browser = webdriver.Chrome(chrome_options=options)

        #browser = webdriver.Chrome()
        browser.get('https://www.invesco.com.hk/mpf/en/mpf/historical-prices')
        sleep(1)

        #select individual investor
        radioElement = browser.find_element_by_id("Individual Investor")
        browser.execute_script("arguments[0].click();", radioElement)

        #move to the end of the page to click "confirm button"
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        #click confirm button
        confirmElement=browser.find_element_by_class_name('o-button')
        confirmElement.click()

        #select different funds

        from selenium.webdriver.support.ui import Select
        from selenium.webdriver.common.by import By
        from selenium.webdriver.support.ui import WebDriverWait as wait
        from selenium.webdriver.support import expected_conditions as EC

        select = Select(browser.find_element_by_id("fundCode"))
        options = select.options
        for index in range(0, len(options)):
            select.select_by_index(index)

            startdate=wait(browser, 10).until(EC.presence_of_element_located((By.ID, "chart-start-date")))
            startdate.click()
            #select month
            startmonth=Select(wait(browser, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "ui-datepicker-month"))))
            startmonth.select_by_index(0)
            #select year
            startyear=Select(wait(browser, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "ui-datepicker-year"))))
            startyear.select_by_index(0) 
            #select day
            days = browser.find_elements_by_xpath('//a[@class="ui-state-default"]')
            days[0].click()

            excelbutton = browser.find_element_by_xpath('//*[@id="btn-export-to-excel"]').click()

            select = Select(wait(browser, 10).until(EC.presence_of_element_located((By.ID, "fundCode"))))
            options = select.options
        sleep(5)
        browser.quit()
    
    except:
        print("Error occured")
        web_scrap()

In [4]:
web_scrap()

<ipython-input-3-8041fd9a0a37>:16: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(chrome_options=options)


## Rename File

In [5]:
import os 
for count, filename in enumerate(os.listdir(path_dir)):  
    newfilename=filename[:filename.index("-")]
    if(newfilename[:7]!="Invesco"):
        newfilename="Invesco "+newfilename[:filename.index(".")]
    else:
        newfilename=newfilename[:filename.index(".")] 
    oldfilepath=path_dir+"/"+filename
    newfilepath = path_dir+"/"+newfilename+".csv"
    os.rename(oldfilepath, newfilepath)
    


## Concate Fund (daily + monthly)
output as Invesco.csv & Invesco_df_monthly.csv

In [8]:
import pandas as pd
dfs = [] #to store dataframe in loop (for daily)
fund_list=[]
funds_monthly_list=[]#for all funds

for count, filename in enumerate(os.listdir(path_dir)): 
    print(filename)

    fundname=filename[:filename.index(".")]
        
    fund_list.append(fundname)
    filepath=path_dir+"/"+filename
    
    df = pd.read_csv(filepath, skiprows=1, index_col=0, parse_dates=True, dayfirst=True) #set date as index
    df.index.names = ['Valuation Date']
    df.columns = [fundname]
    df.sort_values(by=['Valuation Date'], inplace=True)
    
    #*****************************Daily*****************************
    dfs.append(df)
    #*****************************End of daily*****************************
    
    
    #*****************************Monthly*****************************
    df1=df.copy()
    df1.index=pd.to_datetime(df1.index, format="%y-%m-%d").to_period('M')

    #dataframes
    df_start=df1.groupby(df1.index).head(1)  #first one of the month
    df_end=df1.groupby(df1.index).tail(1) #last one of the month

    df_fund_monthly = df_start.copy()

    for i in range(len(df_start)):
        if(df_end.iloc[i][0]==df_start.iloc[i][0]and i!=0):
            monthly_return = (df_end.iloc[i][0]-df_end.iloc[i-1][0])/df_end.iloc[i-1][0]
        else:
            monthly_return = (df_end.iloc[i][0]-df_start.iloc[i][0])/df_start.iloc[i][0]    

        df_fund_monthly.iloc[i][fundname] = monthly_return
    funds_monthly_list.append(df_fund_monthly)
    #*****************************End of monthly*****************************
    
    
print(dfs)    
dfA = pd.concat(dfs, axis=1)
dfA.to_csv('Invesco.csv')

df_monthly = pd.concat(funds_monthly_list,axis=1)  
df_monthly.to_csv('Invesco_df_monthly.csv')


Invesco Age 65 Plus Fund.csv
Invesco Asian Equity Fund.csv
Invesco Balanced Fund.csv
Invesco Capital Stable Fund.csv
Invesco Core Accumulation Fund.csv
Invesco Global Bond Fund.csv
Invesco Growth Fund.csv
Invesco Guaranteed Fund.csv
Invesco Hang Seng Index Tracking Fund.csv
Invesco Hong Kong and China Equity Fund.csv
Invesco MPF Conservative Fund.csv
Invesco RMB Bond Fund.csv
[                Invesco Age 65 Plus Fund
Valuation Date                          
2017-04-03                       10.0000
2017-04-05                       10.0061
2017-04-06                       10.0005
2017-04-07                        9.9950
2017-04-10                        9.9790
...                                  ...
2021-01-21                       12.1302
2021-01-22                       12.1222
2021-01-25                       12.1590
2021-01-26                       12.1472
2021-01-27                       12.0991

[942 rows x 1 columns],                 Invesco Asian Equity Fund
Valuation Date      